In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pysentimiento import create_analyzer
import pandas as pd
import pickle

2024-04-01 03:28:07.559888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 03:28:07.703148: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 03:28:07.706238: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-01 03:28:07.706259: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
#read in posts and comments
with open("posts.pickle", "rb") as f:
    posts = pickle.load(f)
with open("comments.pickle", "rb") as f:
    comments = pickle.load(f)

In [3]:
#load vader sentiment analyzer
vader_analyzer = SentimentIntensityAnalyzer()

In [4]:
#label posts
for idx, vals in enumerate(posts):
    for key, value in vader_analyzer.polarity_scores(vals["content"]).items():
        posts[idx][f"vader_score_{key}"] = value

In [5]:
#label comments
for idx, vals in enumerate(comments):
    for key, value in vader_analyzer.polarity_scores(vals["content"]).items():
        comments[idx][f"vader_score_{key}"] = value

In [6]:
#load bertweet analyzer
analyzer = create_analyzer(task="sentiment", lang="en")

In [7]:
#label posts
for idx, vals in enumerate(posts):
    for key, value in analyzer.predict(vals["content"]).probas.items():
        posts[idx][f"bert_score_{key}"] = value

In [8]:
#label comments
for idx, vals in enumerate(comments):
    for key, value in analyzer.predict(vals["content"]).probas.items():
        comments[idx][f"bert_score_{key}"] = value

In [9]:
#convert posts and comments to dataframe
posts = pd.DataFrame(posts)
comments = pd.DataFrame(comments)

In [10]:
#function to convert bert softmaxes to 3-pole compound score
def bert_compound(row):
    cat = row[["bert_score_NEG", "bert_score_NEU", "bert_score_POS"]].astype(float).idxmax()
    if cat == "bert_score_POS":
        return row["bert_score_POS"]
    elif cat == "bert_score_NEG":
        return -row["bert_score_NEG"]
    elif cat == "bert_score_NEU":
        if row["bert_score_POS"] >= row["bert_score_NEG"]:
            return 1 - row["bert_score_NEU"]
        else:
            return row["bert_score_NEU"] - 1

In [11]:
#actual conversion
posts["bert_compound"] = posts.apply(lambda x: bert_compound(x), axis=1)
comments["bert_compound"] = comments.apply(lambda x: bert_compound(x), axis=1)


In [12]:
#load emotion analyzer
emotion_analyzer = create_analyzer(task="emotion", lang="en")

loading configuration file config.json from cache at /home/seb/.cache/huggingface/hub/models--finiteautomata--bertweet-base-emotion-analysis/snapshots/c482c9e1750a29dcc393234816bcf468ff77cd2d/config.json
Model config RobertaConfig {
  "_name_or_path": "finiteautomata/bertweet-base-emotion-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sadness": 2,
    "surprise": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "mo

In [13]:
#label posts
posts["emotion"] = posts["content"].apply(lambda x: emotion_analyzer.predict(x))
posts["max_emotion"] = posts["emotion"].apply(lambda x: x.output)

In [14]:
#label emotions
comments["emotion"] = comments["content"].apply(lambda x: emotion_analyzer.predict(x))
comments["max_emotion"] = comments["emotion"].apply(lambda x: x.output)

In [15]:
#write to file
posts.to_csv("posts.csv")
comments.to_csv("comments.csv")